<a href="https://colab.research.google.com/github/fopamesmin/movie-project/blob/main/ligue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install colorama

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_url = "/content/drive/MyDrive/DataSet"

In [8]:
import os
import pandas as pd
import requests as req
from bs4 import BeautifulSoup as BS
import logging

# Constants
sofifa_url = "https://www.ligue1.com/ranking"
COLUMNS = ["position", "team", "points", "PLAYED", "WON", "DRAWN", "LOST", "GF", "Diff"]
# COLUMNS = ["position", "team", "points", "PLAYED", "WON", "DRAWN", "LOST", "GF", "GA", "Diff"]
ID_seasonId = ["2023/2024"]

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

logging.basicConfig(level=logging.INFO)
dataset_url = "/content/drive/MyDrive/DataSet"
# Ensure the directory exists
os.makedirs(dataset_url, exist_ok=True)


for season in ID_seasonId:
    players_list = []

    response = req.get(sofifa_url, headers=headers)

    if response.status_code == req.codes.ok:
        soup = BS(response.content, "lxml")
        ul_list = soup.select("div.classement-table-body ul")

        for ul in ul_list:
            li_list = ul.find_all("li", class_="GeneralStats-row")

            for li in li_list:
                player = []
                position = li.find("div", class_="GeneralStats-item--position").text.strip()
                team = li.find("div", class_="GeneralStats-item--club").find("span", class_="desktop-item").text.strip()
                points = li.find("div", class_="GeneralStats-item--points").text.strip()
                played = li.find_all("div", class_="GeneralStats-item")[3].text.strip()
                won = li.find_all("div", class_="GeneralStats-item RankPage-mobileHide")[1].text.strip()
                drawn = li.find_all("div", class_="GeneralStats-item RankPage-mobileHide")[2].text.strip()
                lost = li.find_all("div", class_="GeneralStats-item RankPage-mobileHide")[3].text.strip()
                gf = li.find_all("div", class_="GeneralStats-item RankPage-mobileHide")[4].text.strip()
                # ga = li.find_all("div", class_="GeneralStats-item RankPage-mobileHide")[5].text.strip()
                diff = li.find_all("div", class_="GeneralStats-item")[10].text.strip()

                # player.extend([position, team, points, played, won, drawn, lost, gf, ga, diff])
                player.extend([position, team, points, played, won, drawn, lost, gf, diff])
                players_list.append(player)

    else:
        logging.error(f"Failed to retrieve data for season {season}")

    df = pd.DataFrame(players_list, columns=COLUMNS)

    # df.read()
    # df.to_csv(path.join(dataset_url+".csv"),header=True, index=False)
    #df.to_csv(os.path.join(dataset_url, f"{season}.csv"), header=True, index=False)
    csv_file_path = os.path.join(dataset_url, f"{season.replace('/', '-')}.csv")
    df.to_csv(csv_file_path, header=True, index=False)

# Example read from an existing CSV
#df_20 = pd.read_csv("/content/drive/MyDrive/DataSet/2021.csv")



print(f"Data for season {season}:\n",df)





Data for season 2023/2024:
    position                    team points PLAYED WON DRAWN LOST  GF Diff
0         1     PARIS SAINT-GERMAIN     76     34  10     2   81  33     
1         2               AS MONACO     67     34   7     7   68  42     
2         3       STADE BRESTOIS 29     61     34  10     7   53  34     
3         4              LOSC LILLE     59     34  11     7   52  34     
4         5                OGC NICE     55     34  10     9   40  29     
5         6      OLYMPIQUE LYONNAIS     53     34   5    13   49  55     
6         7                 RC LENS     51     34   9    11   45  37     
7         8  OLYMPIQUE DE MARSEILLE     50     34  11    10   52  41     
8         9          STADE DE REIMS     47     34   8    13   42  47     
9        10        STADE RENNAIS FC     46     34  10    12   53  46     
10       11             TOULOUSE FC     43     34  10    13   42  46     
11       12  MONTPELLIER HÉRAULT SC     41     34  12    12   43  48     
12       1

In [10]:
#transformation
df.drop(columns=['Diff'], inplace=True)




In [11]:
#Création des Mesures et Calculs Statistiques

# calcule du Pourcentage de Victoires.
nombre_victoires = df['WON'].astype(int).sum()  # Somme des victoires
nombre_matchs_joues = df['PLAYED'].astype(int).sum()  # Somme des matchs joués

pourcentage_victoires = (nombre_victoires / nombre_matchs_joues) * 100

print("Pourcentage moyen de victoires :", pourcentage_victoires)

Pourcentage moyen de victoires : 26.47058823529412


In [12]:
# Supposons que 'drawn' contient le nombre de matchs nuls et 'played' contient le nombre de matchs joués
nombre_matchs_nuls = df['DRAWN'].astype(int).sum()  # Somme des matchs nuls
nombre_matchs_joues = df['PLAYED'].astype(int).sum()  # Somme des matchs joués

pourcentage_matchs_nuls = (nombre_matchs_nuls / nombre_matchs_joues) * 100

print("Pourcentage moyen de matchs nuls :", pourcentage_matchs_nuls)

Pourcentage moyen de matchs nuls : 36.76470588235294
